# Libs

In [1]:
import time
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

from enum import Enum
from IPython.display import HTML
from typing import Callable

SEED = 50
np.random.seed(SEED)

%matplotlib notebook

# Map

In [2]:
class ItemType(Enum):
    EMPTY = 0
    OBSTACLE = 3
    TARGET = 2
    START = 1

class Map:
    def __init__(self, shape: tuple[int, int] = (10, 10), resolution: int = 1, name: str = 'Map') -> None:
        self.shape = np.array(shape) * resolution
        self.map = np.zeros(self.shape)
        self.name = name
        self.obstacles_pos = []
    
    def draw(self, ax: plt.Axes):
        ax.set_title("Map")
        ax.grid(visible=True, which='both', axis='both', color='gray', linestyle='--')
        # ax.set_xticks(np.arange(0, 10, 1))
        # ax.set_yticks(np.arange(0, 10, 1))
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        return ax.imshow(self.map, cmap="Blues", interpolation='nearest')
    
    def draw_cost_map(self, ax: plt.Axes):
        ax.set_title("Cost Map")
        ax.grid(visible=True, which='both', axis='both', color='gray', linestyle='--')
        # ax.set_xticks(np.arange(0, 10, 1))
        # ax.set_yticks(np.arange(0, 10, 1))
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        return ax.imshow(self.get_cost_map(), cmap="magma", interpolation='nearest')

    def reset_map(self) -> None:
        self.map = np.zeros(self.shape)

    def add_item(self, x: int, y: int, value: ItemType = ItemType.OBSTACLE) -> None:
        if x < 0 or y < 0 or x >= self.shape[0] or y >= self.shape[1]:
            return
        
        if self.map[y, x] != ItemType.EMPTY.value:
            return

        self.map[y, x] = value.value

    def get_euclidean_distance(self, a: tuple[int, int], b: tuple[int, int]) -> float:
        """Calculates the euclidean distance between two points

        Args:
            a (tuple[int, int]): First point
            b (tuple[int, int]): Second point

        Returns:
            float: Euclidean distance
        """
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    

    def set_goal(self, x: int, y: int) -> None:
        self.add_item(x, y, ItemType.TARGET)
        self.goal = np.array((x, y))

    def get_cost_map(self):
        cost_map = np.zeros(self.shape)
        for i in range(cost_map.shape[0]):
            for j in range(cost_map.shape[1]):
                if not self.has_obstacle(i, j) and not self.has_target(i, j):
                    cost_map[i, j] = self.get_euclidean_distance(self.goal, (i, j))

        return cost_map

    def set_start(self, x: int, y: int) -> None:
        self.add_item(x, y, ItemType.START)

    def set_square_wall(self, x: int, y: int, width: int, height: int) -> None:
        self.add_item(x, y, ItemType.OBSTACLE)
        for i in range(width):
            for j in range(height):
                self.add_item(x+i, y+j)
        
        self.obstacles_pos.append((x, y, width, height))
    
    def check_collision(self, x: int, y: int) -> bool:
        if x < 0 or y < 0 or x >= self.shape[0] or y >= self.shape[1]:
            return True
        
        return self.map[x, y] == ItemType.OBSTACLE.value

    def has_obstacle(self, x: int, y: int, last_pos: tuple[int, int] = None) -> bool:
        # Direct collision
        if self.check_collision(x, y):
            return True
        
        # Passing through an obstacle
        if last_pos is not None:
            for obstacle in self.obstacles_pos:
                if obstacle[1] < last_pos[1] == obstacle[1] >= y \
                    and obstacle[0] <= x and obstacle[0] + obstacle[2] >= x:
                    return True
                        
        
        return False
    
    def has_target(self, x: int, y: int) -> bool:
        return self.map[y, x] == ItemType.TARGET.value

    def __str__(self) -> str:
        return str(self.map)
    

# Agent

In [3]:
class Agent:
    def __init__(self, intial_pos: tuple[float, float] = (0, 0), initial_velocity: tuple[float, float] = (0, 0),
                 weight: float = 0.5, c1: float = 0.5, c2: float = 0.5, id = 0, map_shape: tuple[int, int] = (10, 10),
                 speed_limit: float = 0.8) -> None:
        # Current position and speed
        self.X = np.array(intial_pos, dtype=np.float16)
        self.V = np.array(initial_velocity, dtype=np.float16)

        # Inertial weight
        self.w = weight

        # Personal and social constants
        self.c1 = c1
        self.c2 = c2

        # Personal best position
        self.pbest = np.random.rand(2)

        # Cost function
        self.cost = None

        # Simulation parameters
        self.id = id
        self.map_shape = map_shape
        self.target_reached = False
        self.speed_limit = speed_limit

    
    def get_euclidean_distance(self, a: tuple[int, int], b: tuple[int, int]) -> float:
        """Calculates the euclidean distance between two points

        Args:
            a (tuple[int, int]): First point
            b (tuple[int, int]): Second point

        
        Returns:
            float: Euclidean distance
        """
        return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)
    
    def has_exceeded_limits(self, position: tuple[int, int], last_position: tuple[int, int] = None, has_obstacle: Callable[[int, int], bool] = None) -> bool:
        # Exceeded the limits of the map
        if position[0] < 0 or position[1] < 0 or position[0] >= self.map_shape[0] or position[1] >= self.map_shape[1]:
            return True
        
        # Step into or pass through an obstacle
        if has_obstacle is not None:
            return has_obstacle(int(position[0]), int(position[1]), last_position)
        
        return False

    def update_position(self, has_obstacle: Callable[[int, int], bool] = None,
                        has_target: Callable[[int, int], bool] = None) -> None:
        next_pos = self.X + self.V

        if self.has_exceeded_limits(next_pos, self.X.astype(int), has_obstacle):
            print(f"Agent {self.id} hit an obstacle")
            # Reduce velocity and change randomly the direction TODO
            self.V = 0.01 * self.V

        elif has_target is not None and has_target(int(next_pos[0]), int(next_pos[1])):
            print(f"Agent {self.id} reached the target")
            self.X = next_pos
            self.V = 0.01 * self.V
            self.target_reached = True
        else:
            self.X = next_pos
  
    def update_velocity(self, gbest: tuple[int, int]) -> None:
        # Randomly set r1 and r2
        r1 = np.random.rand()
        r2 = np.random.rand()
        self.V = self.w * self.V + self.c1 * r1 * (self.pbest - self.X) + self.c2 * r2 * (gbest - self.X)

        # Limit velocity
        self.V[0] = self.V[0] if np.abs(self.V[0]) < self.speed_limit else self.speed_limit*np.sign(self.V[0])
        self.V[1] = self.V[1] if np.abs(self.V[1]) < self.speed_limit else self.speed_limit*np.sign(self.V[1])

    def update_pbest(self, signal_distance: float, global_cost: float, gbest: tuple[int, int]) -> None:
        # Handle first assignment of agent cost
        if self.cost is None:
            self.cost = signal_distance
            self.pbest = self.X
            # Handle first assignment of global cost
            if global_cost is None:
                global_cost = self.cost
                gbest = self.X
            # Handle global best improvement in the first agent cost assignment
            elif self.cost < global_cost:
                gbest = self.X
                global_cost = self.cost
        # Handle agent cost improvement
        elif signal_distance < self.cost:
            self.pbest = self.X
            self.cost = signal_distance
            # print(f"Agent {self.id} improved. Cost: {self.cost:.2f} Pos: {self.pbest}")
            # Handle global best improvement
            if self.cost < global_cost:
                gbest = self.X
                global_cost = self.cost
                # print(f"Global best improved. Cost: {global_cost:.2f} Pos: {gbest}")

        return gbest, global_cost

    def step(self, gbest: tuple[float, float], signal_pos: tuple[float, float], global_cost: float,
             has_obstacle: Callable[[int, int], bool] = None, has_target: Callable[[int, int], bool] = None) -> float:
        self.update_velocity(gbest)
        self.update_position(has_obstacle, has_target)
        
        # signal_distance handler will be substituted by the sensor data in the future
        signal_distance = self.get_euclidean_distance(signal_pos, self.X)
        
        return self.update_pbest(signal_distance, global_cost, gbest)

    def draw(self, ax: plt.Axes):
        return ax.plot(self.X[0], self.X[1], color='purple', marker='o')[0]

# Simulation

In [4]:
class Simulation:
    def __init__(self, maze: Map, agents: list[Agent]) -> None:
        self.maze = maze
        self.agents = np.array(agents, dtype=object)
        self.points = np.asarray([[None, None]] * self.agents.shape[0], dtype=object)
        self.global_cost = None
        self.gbest = np.array((0, 0))
        self.gbest_draw = [None, None]
        self.step_one_mode = False

        for i in range(self.agents.shape[0]):
            self.agents[i].id = i
    
    def shuffle_agents(self) -> None:
        self.agents = np.random.permutation(self.agents)
        self.points = np.array([[self.points[agent.id][0], self.points[agent.id][1]] for agent in self.agents])

    def update_map_points(self, index= -1, is_cost_map: bool = False) -> None:
        map_index = int(is_cost_map)
        if index < 0:
            for i, agent in enumerate(self.agents):
                if self.points[i][map_index] is not None:
                    self.points[i][map_index].set_data(agent.X[0], agent.X[1])
        else:
            if self.points[index][map_index] is not None:
                self.points[index][map_index].set_data(self.agents[index].X[0], self.agents[index].X[1])

    def step_all(self) -> None:
        self.shuffle_agents()

        self.update_map_points(is_cost_map=False)
        self.update_map_points(is_cost_map=True)
        
        for i, agent in enumerate(self.agents):
            self.gbest, self.global_cost = agent.step(self.gbest, self.maze.goal, self.global_cost)

    def step_one(self) -> None:
        index = np.random.randint(0, len(self.agents))

        self.update_map_points(index=index, is_cost_map=False)
        self.update_map_points(index=index, is_cost_map=True)

        self.gbest, self.global_cost = self.agents[index].step(self.gbest, self.maze.goal, 
                                                               self.global_cost, self.maze.has_obstacle)

    def animate(self, i: int) -> None:
        
        if self.step_one_mode:
            self.step_one()
        else:
            self.step_all()

        if self.gbest_draw[0] is not None:
            self.gbest_draw[0].set_offsets(self.gbest)

        if self.gbest_draw[1] is not None:
            self.gbest_draw[1].set_offsets(self.gbest)
        
        #TODO: Add stop condition
        # if self.global_cost < 0.01:
        #     print("Global cost below 0.01. Stopping simulation...")
        #     self.anim.pause()

        return [*self.points, self.gbest_draw]

    def draw(self, ax: plt.Axes, is_cost_map: bool = False) -> None:
        if is_cost_map:
            self.maze.draw_cost_map(ax)
        else:
            self.maze.draw(ax)

        index = int(is_cost_map)
        for i, agent in enumerate(self.agents):
            point = agent.draw(ax)
            self.points[i][index] = point

        ax.scatter(self.maze.goal[0], self.maze.goal[1], color='red', marker='x')
        self.gbest_draw[index] = ax.scatter(self.gbest[0], self.gbest[1], color='green', marker='x')
        ax.set_xlim(-1, self.maze.shape[0]+1)
        ax.set_ylim(-1, self.maze.shape[1]+1)

    def run(self, epochs: int = 300, step_one_mode: bool = False) -> None:
        self.step_one_mode = step_one_mode
        fig, ax = plt.subplots(figsize=(12, 10), ncols=2, nrows=1)
        self.draw(ax[0])
        self.draw(ax[1], is_cost_map=True)
        fig.tight_layout()
        print("Simulating...")
        self.anim = animation.FuncAnimation(
            fig,
            self.animate,
            frames=epochs,
            interval=100,
            blit=True
        )

        self.anim.save('animation.gif', fps=10)
        # HTML(anim.to_jshtml(fps=10))
        plt.show()

    def run_raw(self, epochs: int = 300) -> None:
        for i in range(epochs):
            self.animate(i)
            time.sleep(0.5)

    def display(self, figsize: tuple[int, int] = (10, 10)) -> None:
        fig, ax = plt.subplots(figsize=figsize)
        self.draw(ax)
        fig.tight_layout()

In [76]:
class PSO:
    def __init__(self, num_particles: int = 10, c1 = 0.5, c2 = 0.5, speed_limit: int = 1,
                 position_scaller: int = 1, speed_scaller: int = 1, cost_function: Callable = None,
                 bubble_radius: int = 1, swarm_radius: int = 0.25) -> None:
        
        # Classic PSO parameters
        self.num_particles = num_particles
        self.cost_function = cost_function
        self.c1 = c1
        self.c2 = c2

        # Alternative PSO parameters
        self.initialize_aor_variables()

        # Initializations
        self.initialize_positions(scaller=position_scaller)
        self.initialize_velocities(scaller=speed_scaller)

        # Global control variables
        cost = [self.cost_function(x) for x in self.X]
        self.gbest = self.pbest.copy()[np.argmin(cost)]

        # History for plotting
        self.initialize_history()

        # Simulation control variables
        self.speed_limit = speed_limit
        self.walls = []
        self.bubble_radius = bubble_radius
        self.swarm_radius = swarm_radius

    def get_history(self) -> tuple[np.ndarray, np.ndarray, np.ndarray]:
        return np.array(self.X_history), np.array(self.pbest_history), np.array(self.gbest_history), np.array(self.V_history)

    def initialize_history(self) -> None:
        self.X_history = []
        self.V_history = []
        self.pbest_history = []
        self.gbest_history = []

    def initialize_positions(self, scaller: int = 1) -> None:
        self.X = np.random.rand(self.num_particles, 2) * scaller -1
        self.pbest = self.X.copy()

    def initialize_velocities(self, scaller: int = 1) -> None:
        self.V = np.random.rand(self.num_particles, 2) * scaller 

    def initialize_aor_variables(self) -> None:
        # theta -> Angle of rotation (AoR)
        self.theta = np.zeros(self.num_particles, dtype=np.float16) 
        # s -> Dynamic change step for AoR
        self.s = np.array([1 if np.random.rand() > 0.5 else -1 for _ in range(self.num_particles)], dtype=np.float16)   
        # dc -> Probability of changing s each interaction
        self.dc = np.zeros(self.num_particles, dtype=np.float16)
        # temperature -> Prevent theta decreasing too quickly
        self.temperature = np.zeros(self.num_particles, dtype=np.float16)
        # Trr -> Temperature reduction rate
        self.Trr = np.zeros(self.num_particles, dtype=np.float16)

        # Constants
        self.k = 0.01
        self.p = 0.9

    def has_collision_with_particle(self, pos: tuple[int, int]) -> bool:
        for i, agent in enumerate(self.X):
            if np.linalg.norm(agent - pos) < self.swarm_radius:
                return True
        return False
    
    def has_collision_with_wall(self, pos: tuple[int, int]) -> bool:
        bubble = (pos[0], pos[1], self.bubble_radius, self.bubble_radius)
        for wall in self.walls:
            # if pos[0] > wall[0] and pos[0] < wall[0] + wall[2] and pos[1] > wall[1] and pos[1] < wall[1] + wall[3]:
            #     return True
            if bubble[0] < wall[0] + wall[2] and bubble[0] + bubble[2] > wall[0] \
                and bubble[1] < wall[1] + wall[3] and bubble[1] + bubble[3] > wall[1]:
                return True
        return False
    
    def update_velocity(self, index) -> None:
        # Randomly set r1 and r2
        r1 = np.random.rand()
        r2 = np.random.rand()

        # Update velocity as classic PSO [2]
        self.V[index] = self.V[index] + self.c1 * r1 * (self.pbest[index] - self.X[index]) + self.c2 * r2 * (self.gbest - self.X[index])

        # Limit velocity between arbitrary limits
        if np.abs(self.V[index][0]) > self.speed_limit:
            self.V[index][0] = np.sign(self.V[index][0])
        elif np.abs(self.V[index][1]) > self.speed_limit:
            self.V[index][1] = np.sign(self.V[index][1])

        # Update velocity using AoR concept [7]
        RV = [[np.cos(self.theta[index]), -np.sin(self.theta[index])], [np.sin(self.theta[index]), np.cos(self.theta[index])]]
        self.V[index] = np.matmul(RV, self.V[index])

        # Update theta with temperature [8]
        self.theta[index] = self.temperature[index] * self.theta[index]

        # Update temperature [9]
        self.temperature[index] = self.Trr[index] * self.temperature[index]

    def update_position(self, index) -> None:
        next_pos = self.X[index] + self.V[index]

        if self.has_collision_with_wall(next_pos) or self.has_collision_with_particle(next_pos):
            # Change theta [4]
            self.theta[index] += + self.s[index]

            # Set temperature to maximun
            self.temperature[index] = 1.0

            # Increase Dc [5]
            self.dc[index] *= + self.k

            # Randomly modify s and dc (Maybe add a scaller here)
            if np.random.rand() < self.dc[index]:
                print(f"Agent {index} modified. Cost: {self.cost_function(self.X[index]):.2f} Pos: {self.X[index]}")
                self.s[index] *= -1
                self.dc[index] = 0.0

            # Blocking condition to not pass through walls or particles
            next_pos = self.X[index]
        else:
            # Decrease dc [6]
                self.dc[index] *= self.p

        self.X[index] = next_pos

    def evaluate_epoch(self, index: int = None, epoch: int = 0) -> None:
        if index is None:
            return
        
        pbest_cost = self.cost_function(self.pbest[index])
        local_cost = self.cost_function(self.X[index])

        if local_cost < pbest_cost:
            # print(f"Agent {index} improved. Cost: {self.cost_function(self.X[index]):.2f} Pos: {self.X[index]}")
            self.pbest[index] = self.X.copy()[index]
            
            global_cost = self.cost_function(self.gbest)
            if pbest_cost < global_cost:
                print(f"Epoch {epoch}. Global best improved. Cost: {self.cost_function(self.gbest):.2f} Pos: {self.gbest}")
                self.gbest = self.pbest[index]

    def register_history(self) -> None:
        self.X_history.append(self.X.copy())
        self.V_history.append(self.V.copy())
        self.pbest_history.append(self.pbest.copy())
        self.gbest_history.append(self.gbest)

    def run(self, epochs: int = 300, end_condition_checker: Callable = None, timeout_epochs: int = 1000) -> None:      
        try:
            MAX_EPOCHS = timeout_epochs
            while epochs or end_condition_checker is not None:
                conter = epochs if end_condition_checker is None else MAX_EPOCHS-timeout_epochs

                for i in range(self.num_particles):
                    self.update_velocity(i)
                    self.update_position(i)
                    self.evaluate_epoch(i, conter)

                self.register_history()

                if end_condition_checker is not None and end_condition_checker(self.gbest, self.pbest):
                    print(f"End condition reached in {conter} epochs. Best cost: {self.cost_function(self.gbest)}")
                    break
                elif end_condition_checker is None:
                    epochs -= 1
                elif end_condition_checker is not None and timeout_epochs > 0:
                    timeout_epochs -= 1
                elif timeout_epochs == 0:
                    print(f"Timeout reached in {conter} epochs. Best cost: {self.cost_function(self.gbest)}")
                    break
                
                

        except KeyboardInterrupt:
            print("Stopped by user")

        except Exception as e:
            print(e)

    def animate(self, i):
        X_history, pbest_history, gbest_history, V_history = self.get_history()
        if i > len(X_history):
            return [*self.X_plots, *self.pbest_plots, self.gbest_plot]
        
        if X_history is None:
            raise Exception("Run simulation first")
        
        for j in range(len(self.X_plots)):
            self.X_plots[j].set_offsets(X_history[i][j])
            self.V_plots[j].set_offsets(X_history[i][j])
            self.V_plots[j].set_UVC(V_history[i][j][0], V_history[i][j][1])

        for j in range(len(self.pbest_plots)):
            self.pbest_plots[j].set_offsets(pbest_history[i][j])

        self.gbest_plot.set_offsets(gbest_history[i])

        self.plot_title.set_text(f"PSO Epoch: {i}")
        
        return [*self.X_plots, *self.pbest_plots, self.gbest_plot]

    def display(self, fig: plt.Figure, ax: plt.Axes, interval: int = 100) -> None:
        self.plot_title = ax.set_title("PSO")
        ax.grid(visible=True, which='both', axis='both', color='gray', linestyle='--')
        ax.set_xlabel('X')
        ax.set_ylabel('Y')

        for wall in self.walls:
            ax.add_patch(plt.Rectangle((wall[0], wall[1]), wall[2], wall[3], color='black', fill=True, alpha=0.5))
        
        pbest_list = self.get_history()[1][0]
        X_list = self.get_history()[0][0]
        gbest = self.get_history()[2][0]
        V_list = self.get_history()[3][0]

        self.X_plots = [
            ax.scatter(x[0], x[1], color='blue', marker='o') for x in X_list
        ]
        
        self.V_plots = [
            ax.quiver(x[0], x[1], v[0], v[1], angles='xy', scale_units='xy', scale=1, width=0.005, alpha=0.3) for x, v in zip(X_list, V_list)
        ]
        
        self.pbest_plots = [
            ax.scatter(pbest[0], pbest[1], color='orange', marker='x', alpha=0.5) for pbest in pbest_list
        ]

        self.gbest_plot = ax.scatter(gbest[0], gbest[1], color='green', marker='x')

        anim = animation.FuncAnimation(
            fig,
            self.animate,
            frames=len(self.X_history),
            interval=interval,
            blit=True
        )

        return anim
    
    def add_virtual_wall(self, x, y, width, height) -> None:
        self.walls.append((x, y, width, height))

In [96]:
TARGET = (5, 25)
COST_TARGET = 0.5
NUM_PARTICLES = 10

def euclidean_distance(a: np.ndarray, b: np.ndarray) -> float:
    return np.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

def cost_function(X: np.ndarray) -> float:
    return euclidean_distance(X, np.array(TARGET))

def end_condition(gbest: np.ndarray, pbest: np.ndarray) -> bool:
    gbest_contition = cost_function(gbest) < COST_TARGET
    costs = np.array([cost_function(x) for x in pbest], dtype=np.float16)
    pbest_condition = np.count_nonzero(costs < COST_TARGET) >= NUM_PARTICLES

    return gbest_contition and pbest_condition

pso = PSO(num_particles=NUM_PARTICLES, c1=0.3, c2=0.7, position_scaller=10, speed_scaller=0.5, cost_function=cost_function,
          bubble_radius=1, swarm_radius=0.01)
pso.add_virtual_wall(0, 15, 15, 5)
pso.run(epochs=50, end_condition_checker=end_condition, timeout_epochs=500)

0
Epoch 1. Global best improved. Cost: 17.76 Pos: [8.40393456 7.5719043 ]
Epoch 1. Global best improved. Cost: 17.52 Pos: [9.20998865 7.99791601]
0
Epoch 2. Global best improved. Cost: 16.95 Pos: [9.02797885 8.53434442]
0
Epoch 3. Global best improved. Cost: 15.75 Pos: [6.89782764 9.3656332 ]
Epoch 3. Global best improved. Cost: 13.67 Pos: [ 6.8986193  11.46341251]
0
0
0
Epoch 6. Global best improved. Cost: 12.47 Pos: [10.00310716 13.57510925]
0
Epoch 7. Global best improved. Cost: 11.76 Pos: [ 5.56581871 13.25740523]
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
Epoch 57. Global best improved. Cost: 11.07 Pos: [ 4.61586775 13.93493654]
0
0
0
0
0
0
0
0
0
0
Epoch 67. Global best improved. Cost: 7.12 Pos: [-2.07714094 24.24192181]
0
0
0
0
Epoch 71. Global best improved. Cost: 4.52 Pos: [ 8.49956746 27.86244006]
0
0
0
0
Epoch 75. Global best improved. Cost: 1.87 Pos: [ 5.77487226 23.29305014]
0
Epoch 76. Global best improved. Cost: 1.4

In [97]:
fig, ax = plt.subplots(figsize=(10, 10))
ax.scatter(5, 25, color='red', marker='x')
ax.set_xlim(-10, 50)
ax.set_ylim(-10, 50)
anim = pso.display(fig, ax, interval=250)

anim.save('pso.gif', writer='pillow')

<IPython.core.display.Javascript object>

# Execution

In [45]:
LENGTH = 30
HEIGHT = 30
NUM_AGENTS = 10
SPEED_LIMIT = 1

maze_shape = (LENGTH, HEIGHT)
maze = Map(maze_shape)
maze.set_start(0, 0)
maze.set_goal(25, 5)
# maze.set_square_wall(0, 5, 10, 2)
# maze.set_square_wall(12, 15, 8, 2)

rand = np.random.rand

agents = [
    Agent([rand()*10, rand()*4], [rand()*2, rand()*2], weight=0.8, c1 = 0.5, c2 = 0.7, 
          map_shape=maze_shape, speed_limit=SPEED_LIMIT, id=i)
    for i in range(NUM_AGENTS)
]

sim = Simulation(maze, agents)

sim.run(epochs=100, step_one_mode=True)

<IPython.core.display.Javascript object>

MovieWriter ffmpeg unavailable; using Pillow instead.


Simulating...


IndexError: list index out of range